In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic_data_classification').getOrCreate()

In [4]:
df = spark.read.csv("titanic.csv", inferSchema=True,
                   header=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [51]:
my_cols = df.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [52]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [65]:
my_final_data = my_cols.na.drop()

In [66]:
from pyspark.ml.feature import VectorAssembler,CountVectorizer,StringIndexer,OneHotEncoder

In [67]:
#StringIndexer will assign a value to each character
gender_indexer = StringIndexer(inputCol='Sex',outputCol="SexIndex")
#A B C
#1 2 3
#One Hot encoder
#Example A
#[1 , 0 , 0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec') 

In [68]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [69]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [70]:
from pyspark.ml import Pipeline

In [71]:
from pyspark.ml.classification import LogisticRegression

In [72]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [73]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [74]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/Users/apple/anaconda/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [75]:
fit_model = pipeline.fit(train_data)

In [77]:
results = fit_model.transform(test_data)

In [78]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [79]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [80]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [81]:
AUC = my_eval.evaluate(results)

In [82]:
AUC

0.7440891385089229